<a href="https://colab.research.google.com/github/dlfrnaos19/rock_scissors_paper_classifier/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob

#dataset is from
https://laurencemoroney.com/datasets.html

In [ ]:
def resize_images(img_path):
    images = glob.glob(img_path + "/*.png")
    target_size = (28,28)
    for img in images:
        old = Image.open(img)
        new = old.resize(target_size, Image.ANTIALIAS)
        new.save(img,"png")
    
    print(len(images), " Images resized.")


In [ ]:
# resize test_set img
resize_images('/content/rps-test-set/paper')
resize_images('/content/rps-test-set/rock')
resize_images('/content/rps-test-set/scissors')

In [ ]:
# resize train_set img
resize_images('/content/rps/paper')
resize_images('/content/rps/rock')
resize_images('/content/rps/scissors')

In [ ]:
#check img
images = glob.glob('/content/rps-test-set/paper' + "/*.png")
Image.open(images[0])

In [ ]:
def load_data(img_path, number_of_data=2891):
    #가위 : 0, 바위 : 1, 보 : 2
    img_size = 28
    color = 4
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype=np.int32).reshape(number_of_data, img_size,img_size,color)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path+'scissors/*.png'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img # 데이터 영역에 이미지 복사
        labels[idx]=0 #가위 : 0
        idx = idx + 1
    
    for file in glob.iglob(img_path+'rock/*.png'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img # 데이터 영역에 이미지 복사
        labels[idx]=1 #바위 : 1
        idx = idx + 1
    
    for file in glob.iglob(img_path+'paper/*.png'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img # 데이터 영역에 이미지 복사
        labels[idx]=2 #보 : 2
        idx = idx + 1
    
    return imgs, labels

train_img_path = '/content/rps/'
test_img_path = '/content/rps-test-set/'
    


In [ ]:
# train img, label
(x_train, y_train) = load_data(train_img_path,2520)

In [ ]:
# test img, label
(x_test, y_test) = load_data(test_img_path, 371)

In [ ]:
# train set normalize
x_train_norm = x_train/255.0

In [ ]:
# test set normalize
x_test_norm = x_test/255.0

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
# define sequence
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3,3),activation='relu', input_shape=(28,28,4)))
model.add(tf.keras.layers.MaxPool2D(2,2))
model.add(tf.keras.layers.Conv2D(32,(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(x_train_norm, y_train, epochs=7)

In [ ]:
test_loss, test_acc = model.evaluate(x_test_norm, y_test, verbose=2)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
# is prediction, label match
idx = 0
print(predictions[idx])
print(predicted_labels[idx])
print(y_test[idx])
plt.imshow(x_test[idx])

In [ ]:
# get wrong prediction list
import random
wrong_predict_list = []
for i, _ in enumerate(predicted_labels):
    
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)

In [ ]:
samples = random.choices(population=wrong_predict_list, k=5)

In [ ]:
for n in samples:
    print("예측확률분포: ", str(predictions[n]))
    print("라벨 ", str(y_test[n])," 예측결과: ", str(predicted_labels[n]))
    plt.imshow(x_test[n], cmap=plt.cm.binary)
    plt.show()